In [ ]:
%%capture
!pip install ffmpeg
!pip install --upgrade git+https://github.com/huggingface/transformers.git accelerate datasets[audio]
!pip install sounddevice
!pip install transformers -U
!pip install ffpyplayer
!apt-get install libportaudio2
!pip install ffmpeg-python
!pip install huggingface_hub
!pip install -q torch transformers accelerate bitsandbytes transformers sentence-transformers faiss-gpu
!pip -q install langchain openai tiktoken chromadb
!pip -q install langchain
!pip install pypdf
!pip install sentence-transformers==2.2.2
!pip install tiktoken
!pip install faiss-cpu

In [ ]:
import pandas as pd
import os
import re
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg
torch.random.manual_seed(0)
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
torch.random.manual_seed(0)
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import LLMChain

In [ ]:
loader = CSVLoader(file_path='/content/final_sundaram_finance_data.csv', source_column="Launch_URL")
data = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
texts = text_splitter.split_documents(data)

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
persist_directory = 'db'
vectordb = Chroma.from_documents(documents=texts,embedding=embeddings,persist_directory=persist_directory)
vectordb.persist()
vectordb = None
vectordb = Chroma(persist_directory=persist_directory,embedding_function=embeddings)
retriever = vectordb.as_retriever(search_kwargs={"k":3})

In [ ]:
retriever.get_relevant_documents("What is School bus and Mini bus?")

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content="SEGMENT_NAME: LCV SCV Finance\nPRACTICE_TITLE1: Dealer and Tripartite relationship paves way for Bangalore LCV's success in LCV SCV Finance\n\n\nBangalore is one among\nthe major cities for quality education with good schools and colleges, a famous\nIT Hub where staff bus operation is happening in abundance. Also the competition\nwas also equally tough in the market for school and staff buses\n\nHow they did it\n\n·\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\nSchool/staff bus is\none asset segment where there are many challenges for the owner\n\no\xa0\xa0\nThey ply only two\ntrips a day- pick up and drop\n\no\xa0\xa0\nThat too short\ndistance within 30 kms\n\no\xa0\xa0\nCan’t be used to\ngenerate revenue, otherwise \n\no\xa0\xa0\nCan’t be used for any\nother purpose\n\n·\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\nHence they demand long\ntenure, Special offers/discount and lesser rate of interest\n\n·\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\nGenerally in metro\nlike Bangalore,\xa0 60% of Sch

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-128k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.8k [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
template = """As an assistant specializing in question-answering tasks, it is crucial to ensure responses are well-structured and complete. When addressing the query provided, it is imperative to deliver a comprehensive response within a 500-word limit. Please adhere to the following guidelines:

1. If the information necessary to answer the question is available within the documents you have uploaded, please provide a detailed answer. Additionally, cite the source URL from which the answer was derived.
2. If the uploaded documents do not contain the information required to answer the question, clearly state that an answer cannot be provided. Emphasize that this decision is based on the absence of relevant information in the provided documents.
3. It is important to avoid basing your answer on any knowledge external to the content of the uploaded documents. Your responses should be solely informed by the information contained within these documents.
4. In instances where the question pertains to you, please respond with: "I am a SF GenAI Assistant, designed to help you answer questions based on the uploaded documents."
If the question ask about you please response I am a SF GenAI Assistant and help you to answer your question based on the uploaded documents.
Question: {question}
Context: {context}
Answer:
=======
"""
prompt=ChatPromptTemplate.from_template(template)
output_parser = StrOutputParser()

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=500,
)
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
from langchain.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(template)
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
output_parser = StrOutputParser()
from langchain.chains import LLMChain
llm_chain = LLMChain(llm=llm, prompt=prompt)
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [19]:
query="who is the important person in a branch?"
result=rag_chain.invoke(query)
print("******"*20)
print("Answer :")
print(result.split("Answer:")[-1].replace("===",'').strip())
print("Source Url:")
# Extract source links using regular expressions
source_links = re.findall(r'source\': \'(.*?)\'', result)
# Print the extracted source links
list_of_source_url=[]
for link in source_links:
  if(link not in list_of_source_url):
    list_of_source_url.append(link)
print(list_of_source_url)

************************************************************************************************************************
Answer :
=
The important person in a branch, according to the provided context, is the Branch Manager. This conclusion is drawn from several pieces of evidence found across the documents. Firstly, both documents explicitly mention the role of the Branch Manager, stating that "the Branch Manager feels..." indicating their active involvement and opinion on various matters related to the branch's operations. Secondly, the results section attributes the achievement of targets and successful coordination among team members directly to the efforts of the Branch Manager. Lastly, the document detailing activities such as sponsorship at kisan meets, financial transactions involving new versus used tractors, and strategic outreach initiatives like visiting local mandis all point towards decisions and actions taken under the leadership or direction of the Branch Manager. Theref